In [247]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# define the base url needed to create the file url.
base_url = r"https://www.sec.gov"

cik_number = "320193"

# convert a normal url to a document url
normal_url = r"https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/0000320193-20-000096.txt"
normal_url = normal_url.replace('0000320193-20-000096','').replace('.txt','/index.json')

# define a url that leads to a 10k document landing page
documents_url = r"https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/index.json"
print(documents_url)

# request the url and decode it.
content = requests.get(documents_url).json()

for file in content['directory']['item']:
    
    # Grab the filing summary and create a new url leading to the file so we can download it.
    if file['name'] == 'FilingSummary.xml':

        xml_summary = base_url + content['directory']['name'] + "/" + file['name']
        
        print('-' * 100)
        print('File Name: ' + file['name'])
        print('File Path: ' + xml_summary)


https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/index.json
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/FilingSummary.xml


In [248]:
base_url = xml_summary.replace('FilingSummary.xml', '')

# print(base_url)

# request and parse the content
content = requests.get(xml_summary).content
soup = BeautifulSoup(content, 'lxml')         # Format to xml

# print(soup)
# find the 'myreports' tag because this contains all the individual reports submitted.
reports = soup.find('myreports')

# I want a list to store all the individual components of the report, so create the master list.
master_reports = []

# loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
for report in reports.find_all('report')[:-1]:

    # let's create a dictionary to store all the different parts we need.
    report_dict = {}
    report_dict['name_short'] = report.shortname.text
    report_dict['name_long'] = report.longname.text
    report_dict['position'] = report.position.text
    report_dict['category'] = report.menucategory.text
    report_dict['url'] = base_url + report.htmlfilename.text

    # append the dictionary to the master list.
    master_reports.append(report_dict)

    # print the info to the user.
    print('-'*100)
    # if (report.htmlfilename.text == "R11.htm"):
    print(base_url + report.htmlfilename.text)
    print(report.longname.text)
    print(report.shortname.text)
    print(report.menucategory.text)
    print(report.position.text)

----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R1.htm
0001001 - Document - Cover Page
Cover Page
Cover
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R2.htm
1001002 - Statement - CONSOLIDATED STATEMENTS OF OPERATIONS
CONSOLIDATED STATEMENTS OF OPERATIONS
Statements
2
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R3.htm
1002003 - Statement - CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME
CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME
Statements
3
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R4.htm
1003004 - S

In [249]:
# create the list to hold the statement urls
statements_url = []

for report_dict in master_reports:
    
    # define the statements we want to look for.
    item1 = r"CONSOLIDATED STATEMENTS OF OPERATIONS"
    item2 = r"CONSOLIDATED BALANCE SHEETS"
    item3 = r"CONSOLIDATED STATEMENTS OF CASH FLOWS"
    # item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
    
    # store them in a list.
    report_list = [item1, item2, item3]
    
    # if the short name can be found in the report list.
    if report_dict['name_short'] in report_list:
        
        # print some info and store it in the statements url.
        print('-'*100)
        print(report_dict['name_short'])
        print(report_dict['url'])
        
        statements_url.append(report_dict['url'])

----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF OPERATIONS
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R2.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED BALANCE SHEETS
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R4.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF CASH FLOWS
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R7.htm


In [250]:
# let's assume we want all the statements in a single data set.
statements_data = []

# loop through each statement url
for statement in statements_url:

    # define a dictionary that will store the different parts of the statement.
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] = []
    statement_data['data'] = []
    
    # request the statement file content
    content = requests.get(statement).content
    report_soup = BeautifulSoup(content, 'html')

    # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
    for index, row in enumerate(report_soup.table.find_all('tr')):
        
        # first let's get all the elements.
        cols = row.find_all('td')
        
        # if it's a regular row and not a section or a table header
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
            
        # if it's a regular row and a section but not a table header
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            statement_data['sections'].append(sec_row)
            
        # finally if it's not any of those it must be a header
        elif (len(row.find_all('th')) != 0):            
            hed_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['headers'].append(hed_row)
            
        else:            
            print('We encountered an error.')

    # append it to the master list.
    statements_data.append(statement_data)
    print(statement_data)

{'headers': [['CONSOLIDATED STATEMENTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions', '12 Months Ended'], ['Sep. 26, 2020', 'Sep. 28, 2019', 'Sep. 29, 2018']], 'sections': ['Operating expenses:', 'Earnings per share:', 'Shares used in computing earnings per share:'], 'data': [['Net sales', '$ 274,515', '$ 260,174', '$ 265,595'], ['Cost of sales', '169,559', '161,782', '163,756'], ['Gross margin', '104,956', '98,392', '101,839'], ['Research and development', '18,752', '16,217', '14,236'], ['Selling, general and administrative', '19,916', '18,245', '16,705'], ['Total operating expenses', '38,668', '34,462', '30,941'], ['Operating income', '66,288', '63,930', '70,898'], ['Other income/(expense), net', '803', '1,807', '2,005'], ['Income before provision for income taxes', '67,091', '65,737', '72,903'], ['Provision for income taxes', '9,680', '10,481', '13,372'], ['Net income', '$ 57,411', '$ 55,256', '$ 59,531'], ['Basic (in dollars per share)', '$ 3.31', '$ 2.99', '$ 3.00'],

In [251]:
# income_header
(statements_data[0])['headers'][1]

['Sep. 26, 2020', 'Sep. 28, 2019', 'Sep. 29, 2018']

In [252]:
# Grab the proper components
income_header =  (statements_data[0])['headers'][1]
income_data = (statements_data[0])['data']

# Put the data in a DataFrame
income_df = pd.DataFrame(income_data)

# Display
print('-'*100)
print('Before Reindexing')
print('-'*100)
display(income_df.head())

# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0, axis = 1)

# Display
print('-'*100)
print('Before Regex')
print('-'*100)
display(income_df.head())

# Get rid of the '$', '(', ')', and convert the '' to NaNs.
income_df = income_df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)

# Display
print('-'*100)
print('Before type conversion')
print('-'*100)
display(income_df.head())

# everything is a string, so let's convert all the data to a float.
income_df = income_df.astype(float)

# Change the column headers
income_df.columns = income_header

# Display
print('-'*100)
print('Final Product')
print('-'*100)

# show the df
income_df

# drop the data in a CSV file if need be.
# income_df.to_csv('income_state.csv')

----------------------------------------------------------------------------------------------------
Before Reindexing
----------------------------------------------------------------------------------------------------


,0,1,2,3
0,Net sales,"$ 274,515","$ 260,174","$ 265,595"
1,Cost of sales,"169,559","161,782","163,756"
2,Gross margin,"104,956","98,392","101,839"
3,Research and development,"18,752","16,217","14,236"
4,"Selling, general and administrative","19,916","18,245","16,705"


----------------------------------------------------------------------------------------------------
Before Regex
----------------------------------------------------------------------------------------------------


,1,2,3
Category,,,
Net sales,"$ 274,515","$ 260,174","$ 265,595"
Cost of sales,"169,559","161,782","163,756"
Gross margin,"104,956","98,392","101,839"
Research and development,"18,752","16,217","14,236"
"Selling, general and administrative","19,916","18,245","16,705"


----------------------------------------------------------------------------------------------------
Before type conversion
----------------------------------------------------------------------------------------------------


,1,2,3
Category,,,
Net sales,274515,260174,265595
Cost of sales,169559,161782,163756
Gross margin,104956,98392,101839
Research and development,18752,16217,14236
"Selling, general and administrative",19916,18245,16705


----------------------------------------------------------------------------------------------------
Final Product
----------------------------------------------------------------------------------------------------


,"Sep. 26, 2020","Sep. 28, 2019","Sep. 29, 2018"
Category,,,
Net sales,274515.00,260174.00,265595.00
Cost of sales,169559.00,161782.00,163756.00
Gross margin,104956.00,98392.00,101839.00
Research and development,18752.00,16217.00,14236.00
"Selling, general and administrative",19916.00,18245.00,16705.00
Total operating expenses,38668.00,34462.00,30941.00
Operating income,66288.00,63930.00,70898.00
"Other income/(expense), net",803.00,1807.00,2005.00
Income before provision for income taxes,67091.00,65737.00,72903.00


In [253]:
income_df.transpose()



Category,Net sales,Cost of sales,Gross margin,Research and development,"Selling, general and administrative",Total operating expenses,Operating income,"Other income/(expense), net",Income before provision for income taxes,Provision for income taxes,Net income,Basic (in dollars per share),Diluted (in dollars per share),Basic (in shares),Diluted (in shares),Products,Net sales,Cost of sales,Services,Net sales,Cost of sales
"Sep. 26, 2020",274515.0,169559.0,104956.0,18752.0,19916.0,38668.0,66288.0,803.0,67091.0,9680.0,57411.0,3.31,3.28,17352119.0,17528214.0,NaN,220747.0,151286.0,NaN,53768.0,18273.0
"Sep. 28, 2019",260174.0,161782.0,98392.0,16217.0,18245.0,34462.0,63930.0,1807.0,65737.0,10481.0,55256.0,2.99,2.97,18471336.0,18595651.0,NaN,213883.0,144996.0,NaN,46291.0,16786.0
"Sep. 29, 2018",265595.0,163756.0,101839.0,14236.0,16705.0,30941.0,70898.0,2005.0,72903.0,13372.0,59531.0,3.00,2.98,19821510.0,20000435.0,NaN,225847.0,148164.0,NaN,39748.0,15592.0


In [254]:
income_df.T.index

Index(['Sep. 26, 2020', 'Sep. 28, 2019', 'Sep. 29, 2018'], dtype='object')

In [255]:
pd.to_datetime(income_df.T.index)

DatetimeIndex(['2020-09-26', '2019-09-28', '2018-09-29'], dtype='datetime64[ns]', freq=None)

In [256]:
income_df_new = income_df.T
income_df_new.index = pd.to_datetime(income_df_new.index)

income_df_new

Category,Net sales,Cost of sales,Gross margin,Research and development,"Selling, general and administrative",Total operating expenses,Operating income,"Other income/(expense), net",Income before provision for income taxes,Provision for income taxes,Net income,Basic (in dollars per share),Diluted (in dollars per share),Basic (in shares),Diluted (in shares),Products,Net sales,Cost of sales,Services,Net sales,Cost of sales
2020-09-26,274515.0,169559.0,104956.0,18752.0,19916.0,38668.0,66288.0,803.0,67091.0,9680.0,57411.0,3.31,3.28,17352119.0,17528214.0,NaN,220747.0,151286.0,NaN,53768.0,18273.0
2019-09-28,260174.0,161782.0,98392.0,16217.0,18245.0,34462.0,63930.0,1807.0,65737.0,10481.0,55256.0,2.99,2.97,18471336.0,18595651.0,NaN,213883.0,144996.0,NaN,46291.0,16786.0
2018-09-29,265595.0,163756.0,101839.0,14236.0,16705.0,30941.0,70898.0,2005.0,72903.0,13372.0,59531.0,3.00,2.98,19821510.0,20000435.0,NaN,225847.0,148164.0,NaN,39748.0,15592.0


In [257]:
income_df_new = income_df_new.dropna(axis=1)

income_df_new


Category,Net sales,Cost of sales,Gross margin,Research and development,"Selling, general and administrative",Total operating expenses,Operating income,"Other income/(expense), net",Income before provision for income taxes,Provision for income taxes,Net income,Basic (in dollars per share),Diluted (in dollars per share),Basic (in shares),Diluted (in shares),Net sales,Cost of sales,Net sales,Cost of sales
2020-09-26,274515.0,169559.0,104956.0,18752.0,19916.0,38668.0,66288.0,803.0,67091.0,9680.0,57411.0,3.31,3.28,17352119.0,17528214.0,220747.0,151286.0,53768.0,18273.0
2019-09-28,260174.0,161782.0,98392.0,16217.0,18245.0,34462.0,63930.0,1807.0,65737.0,10481.0,55256.0,2.99,2.97,18471336.0,18595651.0,213883.0,144996.0,46291.0,16786.0
2018-09-29,265595.0,163756.0,101839.0,14236.0,16705.0,30941.0,70898.0,2005.0,72903.0,13372.0,59531.0,3.00,2.98,19821510.0,20000435.0,225847.0,148164.0,39748.0,15592.0


In [258]:
# income_df_new.mean(axis=0)

import yfinance as yf

tickers = ["AAPL"] # Tesla stock and Bitcoin in USD

start_date = '2010-01-01'
end_date = '2021-01-01'

In [259]:
panel_data = yf.download(tickers=tickers, start=start_date, end=end_date)[['Adj Close']].dropna(axis=0)
panel_data

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
2010-01-04,6.583586
2010-01-05,6.594968
2010-01-06,6.490066
2010-01-07,6.478067
2010-01-08,6.521136
...,...
2020-12-24,131.773087
2020-12-28,136.486053
2020-12-29,134.668762


In [260]:
# obtain percentage change for each stock to use as the y variable in training
panel_data['AAPL_pct_change'] = panel_data['Adj Close'].pct_change()

# panel_data.dropna(axis=0)

#ignore first row due to NaN
panel_data = panel_data.iloc[1:, :]
panel_data

,Adj Close,AAPL_pct_change
Date,,
2010-01-05,6.594968,0.001729
2010-01-06,6.490066,-0.015906
2010-01-07,6.478067,-0.001849
2010-01-08,6.521136,0.006648
2010-01-11,6.463610,-0.008821
...,...,...
2020-12-24,131.773087,0.007712
2020-12-28,136.486053,0.035766
2020-12-29,134.668762,-0.013315


In [261]:
from datetime import timedelta
# obtain dates of income_df (they are the indices)
dates = income_df_new.index
# subtract one day from each day to obtain the Friday's instead of the saturdays
dates = dates - timedelta(days=1)
# obtain the one year before the minimum date value in financial statement
new_date = dates.min() - timedelta(days=366) # subtract 90 when you do quarterlies
# append new date to our dates
new_dates = np.append(dates, np.datetime64(new_date))
# slice out the desired dates in our pct_change df
panel_data.loc[pd.DatetimeIndex(new_dates), :]

,Adj Close,AAPL_pct_change
2020-09-25,111.919373,0.037516
2019-09-27,53.993587,-0.004866
2018-09-28,54.849937,0.003512
2017-09-27,36.916378,0.007118


In [262]:


# Make new dataframe of relevant stock adjusted close prices (2020-09-26, 2019-09-28, 2018-09-29, 2017-09-29)
consolidated_df = panel_data.loc[new_dates, :]


consolidated_df['AAPL_pct_change'] = consolidated_df[['Adj Close']].pct_change(-1)
consolidated_df = consolidated_df.dropna(axis = 0)

In [263]:
income_df_new['Ann_pct_change'] = consolidated_df['AAPL_pct_change']

income_df_new

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(new_indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

Category,Net sales,Cost of sales,Gross margin,Research and development,"Selling, general and administrative",Total operating expenses,Operating income,"Other income/(expense), net",Income before provision for income taxes,Provision for income taxes,Net income,Basic (in dollars per share),Diluted (in dollars per share),Basic (in shares),Diluted (in shares),Net sales,Cost of sales,Net sales,Cost of sales,Ann_pct_change
2020-09-26,274515.0,169559.0,104956.0,18752.0,19916.0,38668.0,66288.0,803.0,67091.0,9680.0,57411.0,3.31,3.28,17352119.0,17528214.0,220747.0,151286.0,53768.0,18273.0,1.072827
2019-09-28,260174.0,161782.0,98392.0,16217.0,18245.0,34462.0,63930.0,1807.0,65737.0,10481.0,55256.0,2.99,2.97,18471336.0,18595651.0,213883.0,144996.0,46291.0,16786.0,-0.015613
2018-09-29,265595.0,163756.0,101839.0,14236.0,16705.0,30941.0,70898.0,2005.0,72903.0,13372.0,59531.0,3.00,2.98,19821510.0,20000435.0,225847.0,148164.0,39748.0,15592.0,0.485789
